# XENS 2023 : Informatique A

## Partie I - Comptage d'occurrences

### Question I.1

In [1]:
let occurences mot =
  let q = Array.make 256 0 in
  let rec aux list =
    match list with
    | [] -> ();
    | lettre::suite -> q.(lettre) <- q.(lettre) + 1 ; aux suite
  in
  aux mot;
  q;;

val occurences : int list -> int array = <fun>


### Question I.2

In [2]:
let nonzero_occurences occ =
 let n = (Array.length occ) - 1 in 
 let nonzero_count = ref 0 in
 for i = 0 to n do
  if occ.(i) <> 0 then incr nonzero_count;
 done;
 let nonzero_q = Array.make !nonzero_count (0, 0) in
 let index_to_fill = ref 0 in
  for i = 0 to n do
    if occ.(i) <> 0 then
      begin
        nonzero_q.(!index_to_fill) <- (i, occ.(i));
        incr index_to_fill;
      end
    done;
  nonzero_q ;;

val nonzero_occurences : int array -> (int * int) array = <fun>


## Partie II - Arbres Binaires

### Question II.1

On note $S = \{A, B, C, D \}$. Comme $|S| = 4$, un arbre appartenant à $\mathcal{T}_S$ possède exactement $4$ feuilles. Il existe dont $4! = 24$ arbres possibles pour chacune des configurations d'arbres binaires à $4$ feuilles. Or il existe $5$ telles configurations (celle d'un arbre complet de 2 niveaux et les 4 configurations incomplète de 3 niveaux.) 

<img src="attachment:11e5e2b8-964f-4ac4-b18e-dfaa30161bf4.jpg" width="300" />

Il y a donc $5 \times 24 = 5! = 120$ arbres dans $\mathcal{T}_S$

### Question II.2

In [3]:
type tree = F of int | N of tree * tree ;;

let cq arbre occ = 
  let rec aux arbre profondeur = 
    match arbre with
    |F(s) -> occ.(s) * profondeur
    |N(g, d) -> let p = profondeur + 1 in (aux g p) + (aux d p);
  in
  aux arbre 0;;

type tree = F of int | N of tree * tree


val cq : tree -> int array -> int = <fun>


Comme un arbre binaire strict possédant $n$ feuilles possèdent $n - 1$ nœuds internes, la fonction `aux` est exécutée $2n - 1$ fois, puisque cette fonction parcours tout l'arbre par construction. La complexité est donc en $O(n)$.

### Question II.3

In [4]:
let get_path lettre arbre = 
  let path_lettre = ref [] in
  let rec aux arbre path = 
    match arbre with
    |F(x) -> if x = lettre then path_lettre := path
    |N(g, d) -> aux g (0::path) ; aux d (1::path)
  in 
  aux arbre [];
  !path_lettre ;;

val get_path : int -> tree -> int list = <fun>


Cette fonction, dans le pire des cas, parcours l'intégralité de l'arbre avant d'obtenir la suite de bits attendue. La fonction peut donc au moins parcourir tous les noeuds. De plus, comme chaque noeud est parcourue au plus une fois, la complexité de cette fonction est en $O(2n - 1) = O(n)$

### Question II.4

In [5]:
let rec pow a = function
  | 0 -> 1
  | 1 -> a
  | n -> 
    let b = pow a (n / 2) in
    b * b * (if n mod 2 = 0 then 1 else a)
;;

let rec arbre_complet index profondeur =
  if (profondeur <= 0)
  then F(index) 
  else N(arbre_complet index (profondeur - 1), arbre_complet (index + (pow 2 (profondeur - 1))) (profondeur - 1))
;;

let integers borne =
  let rec aux index profondeur=
    if profondeur < borne
    then N(arbre_complet index profondeur, aux (index + pow 2 profondeur) (profondeur + 1))
  else F(index)
  in
  aux 0 0;;

let test = integers 4;;

val pow : int -> int -> int = <fun>


val arbre_complet : int -> int -> tree = <fun>


val integers : int -> tree = <fun>


val test : tree =
  N (F 0,
   N (N (F 1, F 2),
    N (N (N (F 3, F 4), N (F 5, F 6)),
     N
      (N (N (N (F 7, F 8), N (F 9, F 10)),
        N (N (F 11, F 12), N (F 13, F 14))),
      F 15))))


## Partie III - Codes préfixes

### Question III.1

Montrons, par construction de l'arbre $t \in \mathcal{T}_\mathcal{S}$, qu'aucune séquence de bits représentant une lettre n'est préfixe d'une autre séquence de bits représentant une autre lettre.

Soit $\sigma$ une lettre de $\mathcal{S}$, représenté par la séquence de bits $\sigma_b = \sigma_1...\sigma_l$, avec $l \in \mathbb{N}^*$.

Supposons qu'il existe une autre lettre $\sigma' \neq \sigma$ représenté par la séquence de bits $\sigma'_b = \sigma'_1...\sigma'_{l'}$, avec $l' \leq l$ tel que $\sigma'_1...\sigma'_{l'} = \sigma_1...\sigma_{l'}$ i.e que $\sigma'_b$ soit préfixe de $\sigma_b$. Par construction de la séquence de bits depuis la racine pour $\sigma$ et $\sigma'$, alors $F(\sigma')$ est à la fois une feuille et un noeud interne de $t$ -> **absurde**. Donc il n'existe aucun mot possédant une séquence préfixe d'une autre. 

Montrons maintenant que cela implique qu'une séquence de bits soit uniquement décodable, i.e qu'une séquence de bits ne représente qu'au plus un mot de $\mathcal{S^*}$. Soit $b = b_1...b_k$ une séquence de bits.

Comme on vient de montrer qu'aucune séquences de bits représentant une lettre n'est préfixe d'une autre, alors il existe au plus un $0 \leq i \leq k$ tel que $b_1...b_i$ représente une lettre de $t_\mathcal{S}$. De même, si un tel $i$ existe, alors il existe au plus un $i \leq i' \leq l$ tel que $b_i...b_{i'}$ représente une lettre de $t_\mathcal{S}$. Par une récurrence évidente, on trouve bien que $b$ représente au plus un mot de $\mathcal{S^*}$. **D'où le résultat.**

### Question III.2


In [6]:
let decomp1 sequence arbre =
  let rec aux seq arbre_temp res=
    match seq with
    |[] -> if res = [] then failwith "Séquence de bits invalide" else res
    |h::t -> match arbre_temp with
              |F(x) -> aux t arbre (x::res)
              |N(g, d) -> if h = 0 then aux t g res 
                          else if h = 1 then aux t d res 
                          else failwith "L'argument donné n'est pas une séquence de bits"
                        in
  aux sequence arbre [];;


val decomp1 : int list -> tree -> int list = <fun>


## Partie IV - Arbres Optimaux

### Question IV.1

Si on suppose que $g$ n'est pas optimal, alors en faisant la réunion de $S_g$ et $S_d$, on trouve que $c_q(t) > min_{t' \in \mathcal{T}_\mathcal{S}} c_q(t')$, ce qui contredis l'optimalité de $t$, **d'où le résultat**.

### Question IV.2

Si une telle lettre $\sigma_0$ existe, notons $t$ un arbre optimal de $\mathcal{T}_\mathcal{S}$. Alors si $F(\sigma_0)$ est de profondeur $l > 1$ alors $c_q(t)$ = $\Sigma_{\sigma \in \mathcal{S} \setminus \{\sigma_0\}} q(\sigma)$

### Question IV.3

1. 
2. 

### Question IV.4

